In [13]:
#! /usr/bin/env python
"""
Script for generation of input files for imatinib-abl ITC titrations.
"""
from itctools.itctools import ureg, Quantity, compute_rm
from itctools.procedures import ITCProtocol, ITCExperimentSet, ITCExperiment
from itctools.materials import Solvent, Compound, SimpleSolution
from itctools.labware import Labware, PipettingLocation


# Define source solutions on the deck.one
abl_conc = Quantity(5, ureg.millimole/ureg.liter)
ima_conc = Quantity(100, ureg.millimole/ureg.liter)


# Define solvents.
water = Solvent('water', density=0.9970479 * ureg.gram / ureg.milliliter)

# density based on 10 % glyc and water
# http://bit.ly/1i94qPa
buffer = Solvent('buffer', density= 1.02210 * ureg.gram / ureg.milliliter)

# The sample cell volume in microliters
cell_volume = 202.8


# abl MW calculated using peanut 8fba0491a2c1e9ccb8476f4f4fe781bbd1635691
abl = Compound('abl', molecular_weight=33272.95824 * ureg.gram / ureg.mole, purity=1.)
imatinib = Compound(name='imatinib_mesylate', molecular_weight= 589.7084 * ureg.gram / ureg.mole, purity=1.)
imatinib_ka = 1/ Quantity(50, ureg.nanomole/ureg.liter)
# print compute_rm(abl_conc* imatinib_ka).to('dimensionless')

# Define troughs on the instrument.
water_trough = Labware(RackLabel='Water', RackType='Trough 100ml')
buffer_trough = Labware(RackLabel='Buffer', RackType='Trough 100ml')

# Define source labware.
source_plate = Labware(RackLabel='SourcePlate', RackType='5x3 Vial Holder')

abl_solution = SimpleSolution(
    compound=abl,
    compound_mass=abl_conc * abl.molecular_weight * 10 * ureg.milliliter,
    solvent=buffer,
    solvent_mass=10. *
    ureg.gram,
    location=PipettingLocation(
        source_plate.RackLabel,
        source_plate.RackType,
        1))

ima_solution = SimpleSolution(
    compound=imatinib,
    compound_mass=ima_conc * imatinib.molecular_weight * 10 * ureg.milliliter,
    solvent=buffer,
    solvent_mass=10. *
    ureg.gram,
    location=PipettingLocation(
        source_plate.RackLabel,
        source_plate.RackType,
        1))


# Define ITC protocol.

# Protocol for 'control' titrations (water-water, buffer-buffer,
# titrations into buffer, etc.)

binding_protocol = ITCProtocol(
    'binding_protocol',
    sample_prep_method='Plates Quick.setup',
    itc_method='ChoderaAbl.inj',
    analysis_method='Onesite',
    experimental_conditions=dict(target_temperature=25, equilibration_time=300, stir_rate=1000, reference_power=5),
    injections=[dict(volume_inj=0.2, duration_inj=0.4, spacing=100, filter_period=0.5)] +
        25 * [dict(volume_inj=1.5, duration_inj=6, spacing=300, filter_period=0.5)],
    )

blank_protocol = ITCProtocol(
    'blank_protocol',
    sample_prep_method='Plates Quick.setup',
    itc_method='ChoderaAbl.inj',
    analysis_method='Control',
    experimental_conditions=dict(target_temperature=25, equilibration_time=300, stir_rate=1000, reference_power=5),
    injections=[dict(volume_inj=0.2, duration_inj=0.4, spacing=100, filter_period=0.5)] +
        25 * [dict(volume_inj=1.5, duration_inj=6, spacing=300, filter_period=0.5)],
    )


# Define ITC Experiment.
# use specified protocol by default
itc_experiment_set = ITCExperimentSet(name='imatinib_abl')
# Add available plates for experiments.
itc_experiment_set.addDestinationPlate(
    Labware(
        RackLabel='DestinationPlate',
        RackType='ITC Plate'))
itc_experiment_set.addDestinationPlate(
    Labware(
        RackLabel='DestinationPlate2',
        RackType='ITC Plate'))

nreplicates = 1  # number of replicates of each experiment


# # Add buffer control titrations.
# for replicate in range(1):
#     name = 'buffer into buffer %d' % (replicate + 1)
#     itc_experiment_set.addExperiment(
#         ITCExperiment(
#             name=name,
#             syringe_source=buffer_trough,
#             cell_source=buffer_trough,
#             protocol=control_protocol,
#             cell_volume=cell_volume,))

# Abl into buffer.
for replicate in range(1):
    name = 'imatinib into buffer %d' % (replicate + 1)
    itc_experiment_set.addExperiment(
        ITCExperiment(
            name=name,
            syringe_source=ima_solution,
            cell_source=buffer_trough,
            protocol=blank_protocol,
            cell_volume=cell_volume,))

cell_concentration = abl_conc


ima_abl_experiments = list()
buff_abl_experiments = list()
# Scaling factors per replicate
factors = list()
# Define imatinib into abl experiments
for replicate in range(1):
    name = 'imatinib into abl' 
    experiment = ITCExperiment(
        name=name,
        syringe_source=ima_solution,
        syringe_concentration=ima_conc,
        cell_source=abl_solution,
        protocol=binding_protocol,
        cell_volume=cell_volume,
        cell_concentration=abl_conc,
        buffer_source=buffer_trough)

    ima_abl_experiments.append(experiment)

# Define buffer into abl experiments.
for replicate in range(1):
    name = 'buffer into abl'  
    experiment = ITCExperiment(
        name=name,
        syringe_source=buffer_trough,
        cell_source=abl_solution,
        protocol=blank_protocol,
        cell_volume=cell_volume,
        cell_concentration=cell_concentration,
        buffer_source=buffer_trough)
    # rescale to match host into guest experiment concentrations.
    buff_abl_experiments.append(experiment)

# Add buffer to aBL experiment(s) to set
for buff_abl_experiment in buff_abl_experiments:
    itc_experiment_set.addExperiment(buff_abl_experiment)

# Add host to guest experiment(s) to set
for ima_abl_experiment in ima_abl_experiments:
    itc_experiment_set.addExperiment(ima_abl_experiment)
    
# Check that the experiment can be carried out using available solutions
# and plates.
itc_experiment_set.validate(print_volumes=True, omit_zeroes=True)

# Not storing files right now.

# Get effective Rm
actual_Rm = list()
for experiment in itc_experiment_set.experiments:
    try:
        abl_mol = ((experiment.cell_concentration * 202.8 * Quantity('microliter')).to('millimole'))
        ima_mol = ((experiment.syringe_concentration * 30 * Quantity('microliter')).to('millimole'))
        Rm = ima_mol/abl_mol
        actual_Rm.append(Rm)
    except TypeError as e:
        actual_Rm.append(0)

print("#, Final Rm\n")
for idx,act in enumerate(actual_Rm, start=1):
    print("%d, %.5f\n" % (idx, act))



Experiment: 1
 Syringe (ul):120.00  


Experiment: 2
 !!! Buffer   (ul):8.65 
 Transfer (ul):391.35 
 Syringe (ul):120.00  


Experiment: 3
 !!! Buffer   (ul):8.65 
 Transfer (ul):391.35 
 !!! Buffer  (ul):2.59  
 Syringe (ul):120.00  

 Syringe (ul):120.00  


Necessary volumes:
                          Buffer        0.820 mL
                             abl        0.783 mL
               imatinib_mesylate        0.517 mL
Expected waste (3% of total):
                          Buffer        0.025 mL
                             abl        0.023 mL
               imatinib_mesylate        0.016 mL
#, Final Rm

1, 0.00000

2, 0.00000

3, 2.95858

